In [ ]:
#http://www.xicidaili.com/

In [45]:
import requests
import re
import json
import time
from bs4 import BeautifulSoup
from lxml import etree
from requests.exceptions import ReadTimeout,ConnectionError

In [46]:
def cleanString(oldString):
    newString = oldString.replace('\n', '')
    newString = newString.replace('\\', '')
    newString = newString.replace('  ', '')
    newString = newString.replace('   ', '')
    return newString

In [47]:
def saveTotxt(disNum, date_times, user1s, user2s, content_goods, title):
    with open('data/' + title + '.txt', 'a+', encoding='utf-8') as f:
        for i in range(date_times.__len__()):
            f.write(str(disNum) + ',')
            f.write(date_times[i] + ',')
            f.write(user1s[i] + ',')
            f.write(user2s[i] + ',')
            f.write(content_goods[i].replace(',','，') + '\n')
            disNum += 1
        f.close()
    return disNum

In [48]:
def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError:
        return False
    return True

In [49]:
def getDiscussionFromOnePage(url, headers, proxies, disNum, title):
    readTimeout = False
    readTimeout_count = 0
    while True:
        try:
            response = requests.get(url, headers=headers, timeout = 20)
            break
        except ReadTimeout:
            print('Get a timeout')
            readTimeout_count += 1
        except ConnectionError:
            print('Get a timeout')
            readTimeout_count += 1
        if readTimeout_count == 5:
            readTimeout = True
            break
            
    if readTimeout == True:
        print('Always timeout. Shiped current page')
    else:
        if response.status_code == 200 and is_json(response.text):
            jsonData = response.json()
            htmlStr = str(jsonData['data']['html'])
            htmlStr = cleanString(htmlStr)
            html = etree.HTML(htmlStr)
            html_data = html.xpath('//div[@class="list_box"]/div[@class="list_ul"]/div[@class="list_li S_line1 clearfix"]')
            date_times = []
            user1s = []
            user2s = []
            content_goods = []
            for d in html_data:
                date_time = d.xpath('./child::div[@class="list_con"]/div[@class="WB_func clearfix"]/div[@class="WB_from S_txt2"]//text()')[0]
                aNum = d.xpath('./child::div[@class="list_con"]/div[@class="WB_text"]/a').__len__()
                content_bad = d.xpath('./child::div[@class="list_con"]/div[@class="WB_text"]//text()')
                user1 = content_bad[0]
                content_good = ''
                if content_bad.__len__()<4 or aNum<2:
                    for i in range(content_bad.__len__()-1):
                        content_good += content_bad[i+1]
                    user2 = ''
                else:
                    for i in range(content_bad.__len__()-3):
                        content_good += content_bad[i+3]
                    user2 = content_bad[2].replace('@','')
                content_good = content_good.replace('：','')
                content_good = content_good.replace(':','')
                date_times.append(date_time)
                user1s.append(user1)
                user2s.append(user2)
                content_goods.append(content_good)
            return saveTotxt(disNum, date_times, user1s, user2s, content_goods, title)
        elif response.status_code == 414:
            print('Get a 414 on ContentGet')
            time.sleep(30)
            return getDiscussionFromOnePage(url, headers, proxies, disNum, title)
        elif response.status_code == 503:
            print('Get a 503 on ContentGet')
            time.sleep(3)
            return getDiscussionFromOnePage(url, headers, proxies, disNum, title)
        elif response.status_code == 200 and is_json(response.text) == False:
            print('Cookies out time')
            exit()
        else:
            print('Unknow ERROR')
            exit()

In [50]:
def getAUrlByPage(old_url, pageNum):
    url_split = old_url.split('&')
    new_url = ''
    for i in range(url_split.__len__()):
        if url_split[i].find('page=') == -1 and i < url_split.__len__()-1:
            new_url += url_split[i] + '&'
        elif url_split[i].find('page=') == -1 and i == url_split.__len__()-1:
            new_url += url_split[i]
        else:
            new_url += 'page=' + str(pageNum) + '&'
    return new_url

In [53]:
def getWeiboDiscussion(url, disNum, title, currPage):
    proxies = {
        "https":"https://61.128.208.94:3128",
        "https":"https://183.129.207.73:14823",
        "https":"https://223.223.187.195:80",
        "https":"https://203.93.209.163:53281",
        "https":"https://59.37.18.243:3128"
    }
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'Cookie': 'YF-Ugrow-G0=5b31332af1361e117ff29bb32e4d8439; login_sid_t=52e0e65c1dc29400ed0f4325c9351ac5; cross_origin_proto=SSL; YF-V5-G0=572595c78566a84019ac3c65c1e95574; WBStorage=e8781eb7dee3fd7f|undefined; _s_tentry=passport.weibo.com; Apache=4454129403737.665.1540017698992; SINAGLOBAL=4454129403737.665.1540017698992; ULV=1540017699000:1:1:1:4454129403737.665.1540017698992:; wb_view_log=1440*9001; SCF=AkAGBzmnRrBsZyLGJUwLkv_HUqu2cIVaKpTGSpJa2wjbVYDJu2Z238bUfsI674WbT85xk-URhCY1Q6xI5gnSDTs.; SUB=_2A252zr4YDeRhGeNK41cX9S7NzD-IHXVVvajQrDV8PUNbmtBeLXTFkW9NSVAC9jpBb9ite1DdYjv838VnfUISiSp6; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhkzAEoOqAfFKxUuP63.RO75JpX5K2hUgL.Fo-X1h-cSK5pS0e2dJLoIfQLxK-L1K5L1heLxK-L1hML1h5LxKML1-zL1KnLxKBLBonLB-BLxKMLBozLB--LxKqL1KqLBo.LxKBLBonLB.2LxKML12zL1KMLxKBLBonLB-BLxKML12-L12zt; SUHB=0HThLbZ5mYHIGA; ALF=1540622536; SSOLoginState=1540017736; YF-Page-G0=c47452adc667e76a7435512bb2f774f3; wb_view_log_5485650173=1440*9001',
        'Host': 'weibo.com',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36'
    }
    pageNumFlag = False
    try:
        requests.adapters.DEFAULT_RETRIES = 5
        response = requests.get(url, headers=headers, timeout = 20)
        pageNumFlag = True
    except ReadTimeout:
        print('Failed. Get page num failed1')
    except ConnectionError:
        print('Failed. Get page num failed2')
    if pageNumFlag == True and response.status_code == 200 and is_json(response.text):
        jsonData = response.json()
        pageNum = int(jsonData['data']['page']['totalpage'])
        for i in range(pageNum-currPage):
            url = getAUrlByPage(url, i+currPage+1)
            disNum = getDiscussionFromOnePage(url, headers, proxies, disNum, title)
            print(str(i+currPage+1) + ' page finished (Total: ' + str(pageNum) + ')')
        print('Finished')
    elif pageNumFlag == True and response.status_code == 414:
        print('Get a 414 on pageNumGet')
        time.sleep(30)
        return getWeiboDiscussion(url, disNum, title, currPage)
    elif pageNumFlag == True and response.status_code == 503:
        print('Get a 503 on pageNumGet')
        time.sleep(3)
        return getWeiboDiscussion(url, disNum, title, currPage)
    elif response.status_code == 200 and is_json(response.text) == False:
        print('Cookies out time')
        exit()
    else:
        print('Unknow ERROR')
        exit()
    return disNum

In [ ]:
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4140051845624295&root_comment_max_id=199844179840899&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
title = '男生遭女老师扇耳光__'
disNum = 55774
currPage = 3060
disNum = getWeiboDiscussion(url, disNum, title, currPage)

url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4258691002311895&root_comment_max_id=264722606830333&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
title = 'dsadsa__'
disNum = 1
currPage = 0
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4272620234934323&root_comment_max_id=139928885777091&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4278746149699303&root_comment_max_id=145976573366934&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
title = 'asdasd__'
disNum = 1
currPage = 0
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4278370121879682&root_comment_max_id=493422224515708&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)


title = '无印良品工商抽检不配合'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4288150656646566&root_comment_max_id=141029362217570&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4288103877428509&root_comment_max_id=140204715509808&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4288103659307403&root_comment_max_id=142953487552578&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '刘东强性侵事件'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4280867738700611&root_comment_max_id=182535464226416&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4280494268803653&root_comment_max_id=161782160150013&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4280118567961919&root_comment_max_id=165492988248669&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4280086725856896&root_comment_max_id=742049396153077&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '老人地铁车厢打女生'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4294609745896682&root_comment_max_id=176076672664679&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '网红莉哥篡改国歌被行政拘留'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4294931650348135&root_comment_max_id=183223517431517&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4294927137116314&root_comment_max_id=146252439171240&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4293593554408566&root_comment_max_id=188446116624591&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4293599686395048&root_comment_max_id=142953826614236&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '空姐李明珠滴滴打车遇害'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4246109596043656&root_comment_max_id=142401168326183&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4238088089535076&root_comment_max_id=145424335767321&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4238918146416259&root_comment_max_id=154907674638563&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4238419611111695&root_comment_max_id=143087896173824&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4238255018380071&root_comment_max_id=155594828897945&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4237939606703097&root_comment_max_id=163291398095824&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '二十岁女孩乘滴滴顺风车遇害'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4277361954266010&root_comment_max_id=406285866596679&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4277328249115942&root_comment_max_id=191056463357632&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4277075931884995&root_comment_max_id=420579500605389&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4276943160882089&root_comment_max_id=245482265533293&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4277729391903748&root_comment_max_id=154772604079178&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4276937180717705&root_comment_max_id=147900608024691&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '医生称鸿茅药酒是毒药涉损害商誉被抓捕'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4240715925941270&root_comment_max_id=145012179244436&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4230102449794346&root_comment_max_id=158068232282043&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229549748829836&root_comment_max_id=139239071604068&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229463183379374&root_comment_max_id=141575518925467&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229033215940299&root_comment_max_id=140201103519021&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4228805167462203&root_comment_max_id=152158278017689&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4228695302418547&root_comment_max_id=155869122905408&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4228998735208975&root_comment_max_id=157518408855137&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229187974402887&root_comment_max_id=145973548308644&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229484221953476&root_comment_max_id=140613447425795&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229497735923261&root_comment_max_id=154632221323505&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229545194100779&root_comment_max_id=140888331349084&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4240700381824378&root_comment_max_id=151521891848&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4229545194100779&root_comment_max_id=151522483250&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4228825434706177&root_comment_max_id=154219863445685&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)

title = '范冰冰偷税漏税'
disNum = 1
currPage = 0
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4291017299808168&root_comment_max_id=154608376240&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4291110169902050&root_comment_max_id=154689256962&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4291181112267037&root_comment_max_id=1468414942417035&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)
url = 'https://weibo.com/aj/v6/comment/big?ajwvr=6&id=4292880975955735&root_comment_max_id=154809137000&root_comment_max_id_type=&root_comment_ext_param=&page=1&filter=all'
disNum = getWeiboDiscussion(url, disNum, title, currPage)